# Notebook des Résultats des simulations

## Travail sur les conditions initiales

Les premiers résultats de `avg_yy_strain-vs-separation` lors d'un test de traction sans fissure (pour l'instant) n'étaient pas vraiment très concluants. Nous devons donc jouer sur les conditions initiales de la simulation. L'idée est d'obtenir une courbe lisse de *pyy vs ly* sans fissure. Nous obtenons des résultats pas très bons : 

### Jouer sur la vitesse dans le sens de la traction

L'élongation du système augmente de façon non continue. Une idée, serait de diminuer la vitesse des atomes de la partie haute (groupe d'atomes 5) et de la partie mobile (groupe d'atomes 1,2 et 3). On appelle cela **la vitesse d'ouverture**. Cependant, pour avoir des simulations cohérentes, il faut maintenir le produit $\text{vitesse}\times\text{temps simulation}$ constant !

Or : $\text{temps simulation} = \text{nombre pas}\times\text{timestep}$. En dynamique moléculaire, on ne peut pas toucher au timestep car il correspond à la discrétisation temporelle dans le développement de Taylor :

$$\text{for}\quad \Delta t \ll t, \quad X(t + \Delta t) = X(t) + \Delta t \times \dot X(t) + \frac{\Delta t^2}{2} \times \ddot X(t) \quad\text{where}\quad \Delta t = \text{timestep}$$

Donc en diminuant d'un facteur 10 la vitesse d'ouverture, on doit augmenter le nombre de pas d'un facteur 10. Les calculs peuvent donc rapidement devenir conséquant à réaliser sur une machine normale. Pour réaliser les calculs les plus gourmands, je me suis connecté au bastion `dahu` et lancé les calcul dessus. 

<center>

|             | 100 pas - V=0,03 | 300 pas - V=0,01 |
|-------------|:------------------:|:------------------:|
| Produit = 3 |<img src="log_lammps/res/100_pas-0,03V-0,8_bin.png" width="300">|<img src="log_lammps/res/300_pas-0,01V-0,8_bin.png" width="300">|



<center>

|             | 1000 pas - V=0,003 | 3000 pas - V=0,001 |
|-------------|:------------------:|:------------------:|
| Produit = 3 |<img src="log_lammps/res/1000_pas-0,003V-0,8_bin.png" width="300">|<img src="log_lammps/res/3000_pas-0,001V-0,8_bin.png" width="300">|



<center>

|             | 10000 pas - V=0,0003 | 30000 pas - V=0,0001 |
|-------------|:------------------:|:------------------:|
| Produit = 3 |<img src="log_lammps/res/10000_pas-0,0003V-0,8_bin.png" width="300">|<img src="log_lammps/res/30000_pas-0,0001V-0,8_bin.png" width="300">|



On peut remarquer que la vitesse de traction n'a aucune influence sur le lissage de la courbe (même pour un nombre important de run pour une vitese très faible). Il faut donc jouer sur un autre paramètre. 

### Jouer sur le paramètre de voisinage (neighbor list)

#### Qu'est-ce que la "neighborlist" ?

> Source : https://python.plainenglish.io/molecular-dynamics-neighbor-lists-in-python-a8351e6fa3a8

L'un des gros problèmes en **Dynamique Moléculaire** est que la complexité des calculs est en $n^2$ où $n$ est le **nombre d'atomes en simulation**. 

Par exemple, prenons un système avec 100 atomes. Pour calculer les interactions interatomiques, il faut pour chaque atome, calculer les interactions avec les 99 autres. Cela nous fait donc $(100 \text{ atomes}) \times (99 \text{ calculs/atome}) \approx 10^4 \text{ calculs}$. Prenons maintenant un système avec 1000 atomes : $(1000 \text{ atomes}) \times (999 \text{ calculs/atome}) \approx 10^6 \text{ calculs}$. En augmentant d'un facteur $10$ le nombre d'atome, on augmente de $10^2 = 100$ le nombre de calcul ! 

Pour des systèmes beaucoup plus gros comme ceux que j'utilise pour mes simulations, cela peut causer de très lourds calculs qui peuvent de dérouler sur plusieurs jours ou plusieurs semaines. Plusieurs techniques ont donc été expérimentées pour réduire le temps de calcul, comme celle de la "neighbor list". 


##### Neighbor list

Une façon de gérer la croissance quadratique des calculs et de mettre en place un **rayon de coupure** (*cutoff radius* en anglais) : tous les atomes au-delà de ce rayon de coupure sont ignorés pendant les calculs : 

<img src="../rapport/photo/cutoff_radius.png" style="display: block; margin:auto" width="600"/>

Cependant, le choix de la valeur de ce rayon n'est pas trivial. Premièrement, il ne faut pas que les atomes interagissent avec leur image périodique dans le système. Ensuite, implémenter un rayon de coupure influe sur le potentiel interatomique : il ne peut pas être défini en tout point. Cela induit donc des discontinuités dans le potentiel et donc des comportements non voulus. Pour contrer cela, des potentiels prenant en compte le rayon de coupure en paramètre ont été développés. On peut citer le **potentiel de Lennard Jones (LJ)** : 

$$ 
U(r) = 4\epsilon \left[\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6\right]
$$ 

Une fois la valeur du rayon de coupur défini, on peut alors trouver tous les atoms voisins en bouclant sur les positions de chaque atomes :

<img src="../rapport/photo/neighbor_list_visualized.png" style="display: block; margin:auto" width="600"/>

La création de cette liste est toujours un problème en $n^2$ mais elle n'a pas besoin d'être mise à jour à chaque itération. Une méthode pour mettre à jour cette liste voisin est de paramétrer une **épaisseur de peau**. Cette distance est au-delà du rayon de coupure : 

<img src="../rapport/photo/skin_depth.png" style="display: block; margin:auto"  width="200"/>

Dès qu'un atome sort du rayon de coupure **et** de l'épaisseur de peau, cela active la mise à jour de la liste des atomes voisins. De plus, en connaissant la vitesse moyenne ou maximum des atomes, on peut prévoir à quel **timestep** l'atome sortira de cette distance. 


##### Interactions à longue distance

Même si cette méthode de "neighbor list" est facile à implémenter et permet de réduire drastiquement les calculs, il y a un gros désavantage. En effet, cette méthode ne prend pas en compte les intercations à longue distance. Cela rend les simulations bien moins précises. 

Il faut donc choisir soigneusement le rayon de coupure pour que les propriétés ne soient pas très affectées. L'approche pour trouver cette valeur est de diminuer pas à pas le rayon de coupure jusqu'à ce que le système renvoie des valeurs (diffusivité, module élastique, ...) en dehors de l'intervalle d'erreur acceptable : 

<img src="../rapport/photo/acceptable_error.png" style="display: block; margin:auto" width="600"/>

#### Résultats

On a modifié l'**épaisseur de peau** et on a tracé la pression moyenne selon y en fonction de la taille de la boîte selon y : 

<img src="../rapport/photo/pyy-vs-ly_bin.png" style="display: block; margin:auto" width="900"/>

On remarque bien que plus l'épaisseur de peau est faible, plus la courbe se lisse. 

> A confirmer par Noel : c'est normal car la liste des voisins est plus souvent actualisée en diminuant l'épaisseur de peau au-delà du rayon de coupure. Cela actualise donc plus souvent la position des atomes voisins et donc la largeur de la boîte dans la direction y. 